In [41]:
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import json
from pprint import pprint

In [42]:
bb_data = pd.read_csv('../data/bodybuilding_recipes.csv')

In [43]:
bb_data.columns.values

array(['Unnamed: 0', 'name', 'image', 'url', 'mainEntityOfPage',
       'datePublished', 'logo', 'description', 'suitableForDiet',
       'prepTime', 'cookTime', 'totalTime', 'recipeYield', 'servingSize',
       'calories', 'carbohydrates', 'protein', 'fat', 'recipeIngredient',
       'recipeInstructions'], dtype=object)

In [44]:
ingredients = []
for ingredient_list in bb_data.recipeIngredient:
    split_list = ingredient_list.replace('[', '').replace(']','').split(',')
    print(split_list)

["'grilled chicken thighs - 5 lb'", " 'olive oil - 2 tbsp'", " 'rosemary - 2 oz'", " 'lemon juice - ¼ cup'", " 'salt and pepper to taste - '"]
["'chicken thighs", " slice (boneless and skinless) - 6 oz'", " 'mushrooms", " sliced - 2 cup'", " 'poblano pepper - 1 cup'", " 'bacon - 2 slice'", " 'collard greens - 4 cup", " chopped'"]
["'grilled chicken thighs - 6 oz'", " 'broccoli", " chopped - 2 cup'", " 'mushrooms", " sliced - 3 cup'", " 'unsalted butter - 2 tbsp'", " 'white cheddar - 2 oz'", " 'green onion", " chopped - ¼ cup'"]
["'chicken breast", " pounded out to 1/4-inch - 3 piece'", " 'marinara sauce - ½ cup'", " 'pepperoni - 3 oz'", " 'black olives", " sliced - 3 tbsp'", " 'spinach - 1 cup'", " 'fat-free mozzarella cheese - 3 oz'", " 'fresh basil leaves - ½ cup'", " 'salt and pepper to taste - '"]
["'vanilla whey protein powder - ¾ cup(s)'", " 'sugar-free dark chocolate chips - ½ cup'", " 'raw oats - 2 cup'", " 'natural creamy peanut butter - 1 cup'", " 'honey - ¼ cup'", " 'ground 

In [45]:
print(len(ingredients))

0


In [50]:
import json
from pprint import pprint

with open('../data/bodybuilding_recipes.json') as f:
    data = json.load(f)

In [51]:
data[0].keys()

dict_keys(['id', 'contentType', 'locale', 'docType', 'name', 'slug', 'modifiedDate', 'publishDate', 'description', 'cookTime', 'prepTime', 'totalTime', 'servings', 'options', 'directions', 'schemaOrg', 'totalNutrients', 'totalNutrientsPerServing', 'servingSize', '_links', 'lang'])

In [52]:
data[0]['schemaOrg']

{'@context': 'http://schema.org/',
 '@type': 'Recipe',
 'name': 'Everyday Beast Rosemary Grilled Chicken Thighs',
 'image': 'https://www.bodybuilding.com/images/2018/november/everyday-beast-meal-prep-meal-prep-main-header-header.jpg',
 'url': 'https://www.bodybuilding.com/recipes/everyday-beast-rosemary-grilled-chicken-thighs',
 'mainEntityOfPage': True,
 'author': {'@type': 'Person', 'name': ''},
 'datePublished': '11/8/2018',
 'publisher': {'@type': 'Organization',
  'name': 'Bodybuilding.com',
  'logo': {'@type': 'ImageObject',
   'url': 'https://www.bodybuilding.com/images/icons/bb-logo-clean.png',
   'width': 666,
   'height': 422}},
 'description': "This healthy chicken recipe simplifies meal prep and keeps you on track to reach your fitness goals. If you're following a low-carb diet to lose weight, or you're looking for more protein to help you build muscle, this is the recipe for you.",
 'suitableForDiet': [''],
 'prepTime': 'PT60M',
 'cookTime': 'PT10M',
 'totalTime': 'PT70M',

In [107]:
ingredients = data[10]['schemaOrg']['recipeIngredient']
print(len(ingredients))

8


In [108]:
import re


regex = r'[-+]?(?:\d*?[.])?\d+',


for ingredient in ingredients:
    ingredient = ingredient.replace(u"½", u".5")
    ingredient = ingredient.replace(u"¼", u".25")
    split_ingredients = ingredient.split('-')
    this_ingredient = split_ingredients[0].strip()
    
    split_again = split_ingredients[1].split(' ')
    if len(split_again)==3:
        _, amount, unit = split_again
    elif len(split_again)==4:
        _, amount, unit,_ = split_again
    elif len(split_again) == 2:
        amount = 0
        unit = 'N/A'        
    print(this_ingredient, float(amount), unit)
        
    print('')

ribeye steak 290.0 g

olive oil 1.0 tbsp

cream cheese 1.0 tbsp

heavy whipping cream 6.0 tbsp

swiss chard 2.0 cup

mushrooms, sliced 1.0 cup

parmesan cheese, grated 0.25 cup

salt pork 0.5 oz



In [197]:
all_ingredients = []
all_amounts = []
all_units = []
all_descriptors = []
recipe_ids = []
for dat in data:
    recipe_id = dat['id']
    dat = dat['schemaOrg']
    if 'recipeIngredient' in dat.keys():
        ingredients = dat['recipeIngredient']
        for ingredient in ingredients:
            descriptor = None
            recipe_ids.append(recipe_id)

            ingredient = ingredient.replace(u"½", u".5")
            ingredient = ingredient.replace(u"¼", u".25")
            ingredient = ingredient.replace(u"¾", u".75")
            ingredient = ingredient.replace(u"⅛", u".125")
            # print(ingredient)
            split_ingredients = ingredient.split(' -')
            this_ingredient = split_ingredients[0].strip()

            split_again = split_ingredients[1].split(' ')
            if len(split_again)==3:
                _, amount, unit = split_again
            elif len(split_again)==4:
                _, amount, unit,_ = split_again
            elif len(split_again) == 2:
                amount = 0
                unit = None 
            else:
                amount = split_again[1]
                unit = split_again[2]
            #print(ingredient)
            try:
                amount = float(amount)
            except ValueError:
                amount = 0
                unit = None
            
            with_descriptors = this_ingredient.split(',')
            if len(with_descriptors)>1:
                try:
                    this_ingredient = with_descriptors[0]
                    # print(with_descriptors)
                    with_descriptors = with_descriptors[1:]
                    # print(with_descriptors)
                   
                    descriptor = ' '.join(with_descriptors).strip()
                    # print(descriptor)
                    # print('')
                    
                except:
                    print(with_descriptors)
            else:
                descriptor = None
            #print(this_ingredient, float(amount), unit)
        
            all_ingredients.append(this_ingredient.lower())
            all_amounts.append(amount)
            all_units.append(unit)
            if descriptor:
                print('hello', descriptor)
            all_descriptors.append(descriptor)
            #print('')

hello slice (boneless and skinless)
hello sliced
hello chopped
hello sliced
hello chopped
hello pounded out to 1/4-inch
hello sliced
hello quartered and thinly sliced
hello white meat
hello sliced
hello grated
hello cut into quarters
hello raw
hello chopped
hello raw
hello dried
hello raw
hello canned
hello canned
hello minced
hello cut into 1/2-inch strips
hello raw
hello sliced
hello diced
hello chuck for stew  cut into cubes
hello chopped
hello finely diced
hello chopped
hello hardboiled
hello sliced into strips
hello French Vanilla Creme
hello unsweetened
hello unsweetened
hello vanilla
hello chopped
hello minced
hello raw
hello frozen
hello cut into cubes
hello chopped
hello minced
hello grated
hello dried
hello chopped
hello minced
hello chopped
hello melted
hello cut into cubes
hello plain
hello chopped
hello diced
hello plain
hello crushed
hello chopped
hello cut into 1-inch pieces
hello raw
hello frozen
hello frozen
hello frozen
hello plain
hello shredded
hello fresh squeezed


hello slivered
hello chopped
hello about 1 inch thick
hello ground
hello sliced
hello minced
hello baked
hello chopped
hello sliced
hello cubed
hello minced
hello powdered
hello drained and rinsed
hello chopped
hello Triple Chocolate Brownie
hello Triple Chocolate Brownie
hello French Vanilla Creme
hello Triple Chocolate Brownie
hello chopped
hello chopped
hello baked
hello chopped
hello washed and cubed
hello sliced
hello minced
hello sliced
hello ground
hello drained
hello canned
hello dried
hello dried
hello diced
hello minced
hello finely diced
hello canned
hello sliced
hello cooked
hello brown sugar blend
hello sliced
hello sliced
hello minced
hello sliced
hello minced
hello finely diced
hello peeled and chopped
hello finely diced
hello powdered
hello caramel
hello canned in water
hello chopped
hello sliced
hello chopped
hello minced
hello unsweetened
hello unsweetened
hello chocolate peanut butter
hello cold
hello frozen  low fat
hello minced
hello sliced
hello diced
hello draine

hello fried
hello unsweetened
hello diced
hello without salt
hello diced
hello minced
hello chopped
hello diced
hello minced
hello finely diced
hello sliced
hello drained and rinsed
hello shredded
hello minced
hello diced
hello finely chopped
hello diced
hello cooked
hello ground
hello minced
hello sliced
hello diced
hello minced
hello finely chopped
hello diced
hello finely chopped
hello minced
hello minced
hello finely diced
hello ground
hello pieces
hello boiled
hello precooked and diced
hello sliced
hello unsweetened
hello chocolate cookie
hello thinly sliced
hello minced
hello shredded
hello dried
hello minced
hello minced
hello finely chopped
hello diced
hello finely chopped
hello minced
hello KRAFT Mayo Light Mayonnaise
hello bread  dry mix
hello nonfat greek yogurt
hello diced
hello nonfat greek yogurt
hello flaked
hello killa vanilla
hello nonfat greek yogurt
hello vanilla caramel
hello sliced
hello apple pie
hello diced
hello chopped
hello raw  pink and red  Florida
hello bir

In [203]:
len(set(all_descriptors))

228

In [204]:
all_ingredient_set = list(set(all_ingredients))
print(len(all_ingredient_set))

1141


In [205]:
sorted(all_ingredient_set)

['100-calorie multi-grain tortilla',
 '100-calorie popcorn',
 '2% cottage cheese',
 '2% greek yogurt',
 '4 oz tilapia fillets',
 '93% lean ground turkey',
 'adzuki beans',
 'agave syrup',
 'ahi tuna',
 'albacore tuna',
 'all-purpose flour',
 'allspice',
 'almond flour',
 'almond meal',
 'almond milk',
 'almond oil',
 'almonds',
 'anchovy',
 'andouille chicken sausage',
 'andouille sausage',
 'angel food cake',
 'animal chocolate whey',
 'animal vanilla whey',
 'apple',
 'apple cider vinegar',
 'apple juice',
 'apples',
 'apricots',
 'aquafaba',
 'arrowroot flour',
 'arrowroot starch',
 'artichoke',
 'artichoke hearts',
 'artichokes',
 'arugula',
 'asparagus',
 'asparagus spears',
 'avocado',
 'avocado oil',
 'baby carrots',
 'baby corn',
 'baby spinach',
 'bacon',
 'bacon strips',
 'baking chips',
 'baking chocolate',
 'baking powder',
 'baking soda',
 'baking stevia',
 'balsamic vinegar',
 'banana',
 'banana extract',
 'banana or vanilla protein powder',
 'barbecue beans',
 'barbecue 

In [206]:
recipe_ingredient_table = pd.DataFrame({'recipe_id':recipe_ids, 'ingredient':all_ingredients, 'amount':all_amounts, 'descriptor':all_descriptors})


In [207]:
vcs = recipe_ingredient_table['ingredient'].value_counts()

In [208]:
for index, entry in vcs.iteritems():
    print(index, entry)

olive oil 294
garlic 284
egg whites 242
water 232
cinnamon 230
egg 228
nonfat plain greek yogurt 220
salt and pepper to taste 202
vanilla extract 196
banana 163
table salt 156
onion 155
powdered stevia 153
vanilla whey protein powder 151
baking powder 143
raw oats 130
lemon juice 129
chicken breast 127
black pepper 122
honey 112
avocado 100
unsweetened almond milk 100
cocoa powder 97
almond milk 97
coconut flour 94
cilantro 89
natural peanut butter 88
baking soda 87
salt 86
spinach 86
red onion 82
liquid egg whites 82
splenda 81
strawberries 80
cumin 80
tomatoes 77
sweet potato 77
vanilla protein powder 77
skim milk 73
garlic powder 73
rolled oats 72
paprika 70
blueberries 68
coconut oil 67
almonds 65
lime juice 64
ice 64
green onion 58
unsweetened applesauce 55
red bell pepper 55
low-fat parmesan cheese 55
ginger 54
oregano 54
stalk celery 53
light soy sauce 53
fat-free cottage cheese 52
extra virgin coconut oil 52
ground 99% fat-free turkey breast 51
cayenne pepper 50
walnuts 49
pump

ragu sauce 1
tabasco sauce 1
soba noodles 1
trout 1
whole-grain egg noodles 1
paleopro protein 1
wheat chex 1
mini oreos 1
tomatillos 1
gimme vegan lean sausage 1
beast sports creature powder 1
pizza sauce 1
nla for her aminos 1
dymatize elite xt (banana nut) 1
tri-bean blend 1
cornichon finely chopped 1
dymatize elite casein 1
corn flakes 1
hydra 6 1
honeydew melon 1
plain cheerios 1
no-sugar added dark chocolate baking bar with stevia 1
banana extract 1
ribeye steak 1
optimum nutrition amino energy orange cooler 1
jicama 1
soda water 1
roasted beets 1
frozen low-calorie ice cream (vanilla) 1
frozen peas and carrots 1
wasabi powder 1
baking chocolate 1
thai green curry paste 1
shelled edamame 1
hemp protein 1
opti-bar (chocolate brownie) 1
frozen spinach 1
cellucor cor-performance whey (molten chocolate) 1
whole star anise 1
dark chocolate baking powder 1
met-rx vanilla cinnamon protein &amp; oats 1
chocolate almond milk 1
sweet carrots 1
powdered sugar 1
black-eyed peas 1
fat-free le

In [190]:
recipe_ingredient_table['descriptor'].value_counts()

Series([], Name: descriptor, dtype: int64)